# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut
deeplabcut.__file__

/home/jojo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'/home/jojo/DeepLabCut-master/deeplabcut/__init__.py'

In [2]:
videofile_path = ['/home/jojo/DeepLabCut-master/videos/lick_tracking'] 
path_config_file =['/home/jojo/DeepLabCut-master/']

In [3]:
task='lineartrack' # Enter the name of your experiment Task
experimenter='DR' # Enter the name of the experimenter
video=['/home/jojo/DeepLabCut-master/videos/lick_tracking/20191016_Lotus_03_lineartrack.mp4']
deeplabcut.create_new_project(task,experimenter,video, working_directory='working_directory',copy_videos=False) #change the working directory to where you want the folders created.

Created "/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/videos"
Created "/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/labeled-data"
Created "/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/training-datasets"
Created "/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/dlc-models"
Creating the symbolic link of the video
Created the symlink of /home/jojo/DeepLabCut-master/videos/lick_tracking/20191016_Lotus_03_lineartrack.mp4 to /home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/videos/20191016_Lotus_03_lineartrack.mp4
/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/videos/20191016_Lotus_03_lineartrack.mp4
Generated "/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/config.yaml"

A new project with name lineartrack-DR-2019-10-17 is created at /home/jojo/DeepLabCut-master/working_directory and a configurable file (config.

'/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/config.yaml'

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [4]:
%matplotlib inline
path_config_file = '/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/config.yaml'  # Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'manual',crop=True) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!

Quitting for now!

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [5]:
%gui wx
path_config_file = '/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/config.yaml'  # Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [6]:
path_config_file = '/home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/config.yaml'   # Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by DR.
They are stored in the following folder: /home/jojo/DeepLabCut-master/working_directory/lineartrack-DR-2019-10-17/labeled-data/20191016_Lotus_03_lineartrack_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [7]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training - If you want to use a CPU, continue. 
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(path_config_file)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(path_config_file)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
path_config_file='/home/jojo/DeepLabCut-master/working_directory/lineartrack-AJ-2019-04-04/config.yaml'
videofile_path = ['/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_02_lineartrack.mp4','/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_04_lineartrack.mp4','/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_06_lineartrack.mp4']  #Enter the list of videos to analyze
deeplabcut.analyze_videos(path_config_file, videofile_path, save_as_csv=True, videotype='.mp4')

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
path_config_file='/home/jojo/DeepLabCut-master/working_directory/lineartrack-AJ-2019-04-04/config.yaml'
videofile_path = ['/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_02_lineartrack.mp4','/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_04_lineartrack.mp4','/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_06_lineartrack.mp4']  #Enter the list of videos to analyze
deeplabcut.extract_outlier_frames(path_config_file,videofile_path)

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
%gui wx
path_config_file='/home/jojo/DeepLabCut-master/working_directory/lineartrack-AJ-2019-04-04/config.yaml'
videofile_path = ['/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_02_lineartrack.mp4','/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_04_lineartrack.mp4','/home/jojo/DeepLabCut-master/videos/20190320_aj69/20190320_aj69_06_lineartrack.mp4']  #Enter the list of videos to analyze
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path,save_frames=True)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [ ]:
import os.path
#import sys
import argparse, glob, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import subprocess
from pathlib import Path

import matplotlib as mpl
if os.environ.get('DLClight', default=False) == 'True':
    mpl.use('AGG') #anti-grain geometry engine #https://matplotlib.org/faq/usage_faq.html
else:
    mpl.use('TkAgg')
import matplotlib.pyplot as plt

from deeplabcut.utils import auxiliaryfunctions
from deeplabcut.pose_estimation_tensorflow.config import load_config
from skimage.util import img_as_ubyte
from skimage.draw import circle_perimeter, circle
from deeplabcut.utils.video_processor import VideoProcessorCV as vp # used to CreateVideo



In [ ]:
from skimage import io; io.use_plugin('matplotlib')

In [ ]:
y2=480
y1=100
x2=200
x1=0
nframes= 100
start=10

In [ ]:
coords =[x1,x2,y1,y2]
 

In [ ]:
config_file = Path(config).resolve()


	
with open(str(config_file), 'r') as ymlfile:
	  
      cfg = yaml.load(ymlfile)  
      videos = cfg['video_sets'].keys()

In [ ]:
def check_crop(video,start,coords):

    """
    --------
    coord is a list
    start in second
    video is the path of the video?? in " .mp4"

    """
    import os
    import sys
    import yaml
    import numpy as np
    from moviepy.editor import VideoFileClip
    from pathlib import Path
    from skimage import io
    from skimage.util import img_as_ubyte
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches

    from deeplabcut.generate_training_dataset import frameselectiontools


    plt.close("all")
    # update to openCV
    clip = VideoFileClip(video)
    image = clip.get_frame(start)  # frame is accessed by index *1./clip.fps (fps cancels)
    fname = Path(video)
   # output_path = Path(config).parents[0] / 'labeled-data' / fname.stem

    fig ,ax = plt.subplots(1)
    # Display the image
    ax.imshow(image)
    # Create a Rectangle patch
    rect = patches.Rectangle((int(coords[0]),int(coords[2])),int(coords[1]),int(coords[3]),linewidth=3,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    plt.show()

    print("The red boundary indicates how the cropped image will look.")


    # crop and move on with extraction of frames:
    # clip = clip.crop(y1=int(coords[2]), y2=int(coords[3]), x1=int(coords[0]), x2=int(coords[1]))

In [ ]:
check_crop(video,start,coords)

In [ ]:

check_crop('/home/jojo/DeepLabCut-master/video_crop_cut/121318_65_0_CC.mp4',10,coords) 

In [ ]:
def CreateVideo_CutCrop(videos,x1,x2,y1,y2,nframes,videotype='avi',save_frames=True,delete=True):

    """
    videos : list
        A list of string containing the full paths of the videos to analyze.    
    videotype: string, optional
        Checks for the extension of the video in case the input is a directory.\nOnly videos with this extension are analyzed. The default is ``.avi``
    save_frames: bool
        If true creates each frame individual and then combines into a video. This variant is relatively slow as
        it stores all individual frames. However, it uses matplotlib to create the frames and is therefore much more flexible (one can set transparency of markers, crop, and easily customize).
    delete: bool
        If true then the individual frames created during the video generation will be deleted.
   
    Examples
    --------
    If you want to create the labeled video for only 1 video
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/reachingvideo1.avi'])
    --------
    If you want to create the labeled video for only 1 video and store the individual frames
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/reachingvideo1.avi'],save_frames=True)
    --------
    If you want to create the labeled video for multiple videos
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/reachingvideo1.avi','/analysis/project/videos/reachingvideo2.avi'])
    --------
    If you want to create the labeled video for all the videos (as .avi extension) in a directory.
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/'])
    --------
    If you want to create the labeled video for all the videos (as .mp4 extension) in a directory.
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/'],videotype='mp4')
    --------
    """
   
    ''' Creating individual frames and making a video'''
    if [os.path.isdir(i) for i in videos] == [True]:
      print("Analyzing all the videos in the directory")
      videofolder= videos[0]
      os.chdir(videofolder)
      Videos = np.sort([fn for fn in os.listdir(os.curdir) if (videotype in fn)])
      print("Starting ", videofolder, Videos)
    else:
      Videos = videos

    for video in Videos:
        videofolder= Path(video).parents[0] #where your folder with videos is.
        os.chdir(str(videofolder))
        videotype = Path(video).suffix
        print("Starting % ", videofolder, videos)
        vname = str(Path(video).stem)          
        print("Loading ", video, "and data.")

        if save_frames==True:
            tmpfolder = os.path.join(str(videofolder),'temp-' + vname)
            auxiliaryfunctions.attempttomakefolder(tmpfolder)
            clip = vp(video)
            ny, nx= y2-y1,x2-x1 
            fps=clip.fps()
            
            duration = nframes/fps

            print("Duration of video [s]: ", round(duration,2), ", recorded with ", round(fps,2),"fps!")
            print("Overall # of frames: ", int(nframes), "with cropped frame dimensions: ",nx,ny)
            print("Generating frames")
            for index in tqdm(range(nframes)):
                imagename = tmpfolder + "/file%04d.png" % index
                if os.path.isfile(tmpfolder + "/file%04d.png" % index):
                    image = img_as_ubyte(clip.load_frame()) #still need to read (so counter advances!)
                else:
                    plt.axis('off')
                    image = img_as_ubyte(clip.load_frame())                   
                    image=image[y1:y2,x1:x2]
                    
                    plt.figure(frameon=False, figsize=(nx * 1. / 100, ny * 1. / 100))
                    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
                    plt.imshow(image)


                    plt.xlim(0, nx)
                    plt.ylim(0, ny)
                    plt.axis('off')
                    plt.subplots_adjust(
                        left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
                    plt.gca().invert_yaxis()
                    plt.savefig(imagename)

                    plt.close("all")

            start= os.getcwd()
            os.chdir(tmpfolder)

            print("All frames were created, now generating video...")
            vname=str(Path(tmpfolder).stem).split('-')[1]
            try: ## One can change the parameters of the video creation script below:
                subprocess.call([
                    'ffmpeg', '-framerate',
                    str(clip.fps()), '-i', 'file%04d.png', '-r', '30','../'+vname +'_CC.mp4'])
            except FileNotFoundError:
                print("Ffmpeg not correctly installed, see https://github.com/AlexEMG/DeepLabCut/issues/45")

            if delete:
                for file_name in glob.glob("*.png"):
                    os.remove(file_name)
            os.chdir(start)





In [ ]:
CreateVideo_CutCrop(['/home/jojo/DeepLabCut-master/video_crop_cut/121318_65_0.1.h264'],x1,x2,y1,y2,nframes,videotype='h264')

In [ ]:
def CreateVideo_CutCrop(videos,x1,x2,y1,y2,coords,nframes,videotype='avi',save_frames=True,delete=True):

    """
    videos : list
        A list of string containing the full paths of the videos to analyze.    
    videotype: string, optional
        Checks for the extension of the video in case the input is a directory.\nOnly videos with this extension are analyzed. The default is ``.avi``
    save_frames: bool
        If true creates each frame individual and then combines into a video. This variant is relatively slow as
        it stores all individual frames. However, it uses matplotlib to create the frames and is therefore much more flexible (one can set transparency of markers, crop, and easily customize).
    delete: bool
        If true then the individual frames created during the video generation will be deleted.
   
    Examples
    --------
    If you want to create the labeled video for only 1 video
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/reachingvideo1.avi'])
    --------
    If you want to create the labeled video for only 1 video and store the individual frames
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/reachingvideo1.avi'],save_frames=True)
    --------
    If you want to create the labeled video for multiple videos
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/reachingvideo1.avi','/analysis/project/videos/reachingvideo2.avi'])
    --------
    If you want to create the labeled video for all the videos (as .avi extension) in a directory.
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/'])
    --------
    If you want to create the labeled video for all the videos (as .mp4 extension) in a directory.
    >>> deeplabcut.create_labeled_video(['/analysis/project/videos/'],videotype='mp4')
    --------
    """
   
    ''' Creating individual frames and making a video'''
    if [os.path.isdir(i) for i in videos] == [True]:
      print("Analyzing all the videos in the directory")
      videofolder= videos[0]
      os.chdir(videofolder)
      Videos = np.sort([fn for fn in os.listdir(os.curdir) if (videotype in fn)])
      print("Starting ", videofolder, Videos)
    else:
      Videos = videos

    for video in Videos:
        videofolder= Path(video).parents[0] #where your folder with videos is.
        os.chdir(str(videofolder))
        videotype = Path(video).suffix
        print("Starting % ", videofolder, videos)
        vname = str(Path(video).stem)          
        print("Loading ", video, "and data.")

        if save_frames==True:
            tmpfolder = os.path.join(str(videofolder),'temp-' + vname)
            auxiliaryfunctions.attempttomakefolder(tmpfolder)
            clip = vp(video)
            ny, nx= y2-y1,x2-x1 
            fps=clip.fps()
            
            duration = nframes/fps

            print("Duration of video [s]: ", round(duration,2), ", recorded with ", round(fps,2),"fps!")
            print("Overall # of frames: ", int(nframes), "with cropped frame dimensions: ",nx,ny)
            print("Generating frames")
            for index in tqdm(range(nframes)):
                image = clip.get_frame(start)  # frame is accessed by index *1./clip.fps (fps cancels)
                
                fig ,ax = plt.subplots(1)
                # Display the image
                ax.imshow(image)
                # Create a Rectangle patch
                rect = patches.Rectangle((int(coords[0]),int(coords[2])),int(coords[1]),int(coords[3]),linewidth=3,edgecolor='r',facecolor='none')
                # Add the patch to the Axes
                ax.add_patch(rect)
                plt.show()

                print("The red boundary indicates how the cropped image will look.")





In [ ]:
CreateVideo_CutCrop(['/home/jojo/DeepLabCut-master/video_crop_cut/121318_65_0.1.h264'],x1,x2,y1,y2,coords,nframes,videotype='h264',save_frames=True,delete=True)